In [24]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox
from ipywidgets import interactive_output
from IPython.display import clear_output

class GreenAmptModel:
    def __init__(self, h0, theta_0):
        self.h0 = h0
        self.theta_0 = theta_0
        self.reference_lines = []
        self.ref_line_params = []
        self.short_time_ref = []
        self.short_time_params = []
        self.infiltration_rate_ref = []
        self.long_time_approximation_ref = []
        self.ref_line_colors = ['black']


    def t_for_cum_inf(self, i_t, d_theta, d_h, Ks):
        return (i_t - d_theta * d_h * np.log(1+ (i_t/(d_theta*d_h))))/Ks

    def short_time_approximation(self, d_theta, d_h, K, t):
        """Short time approximation for cumulative infiltration
        Parameters
        ----------
        d_theta : float
            Change in volumetric water content
        d_h : float
            Change in pressure head
        K : float
            Hydraulic conductivity
        t : array_like
            Time
        Returns
        -------
        i_t : array
            Cumulative infiltration"""

        return np.sqrt((d_theta*d_h*K)/(2*t))

    def long_time_approximation(self, K, x):
        """Long time approximation for cumulative infiltration
        Parameters
        ----------
        K : float
            Hydraulic conductivity
        t : array_like
            Time
        Returns
        -------
        K : float
            Hydraulic conductivity"""

        it_long = np.array([K]*len(x))

        return it_long

    def d_h(self, h0, hf):
        return h0 - hf

    def d_theta(self, theta_0, theta_i):
        return theta_0 - theta_i
    
    def infiltration_rate(self, new_x, t, I_t):
        y2 = np.interp(new_x, t, I_t)
        diff = np.diff(y2)
        self.diff = diff
        return diff
    

    def plot_scenario_1(self, hf=-60, theta_i=0.2, Ks=4, x_range="0, 200", plot_short_time=True, plot_infiltration_rate=True, plot_long_term=False, i_t_range="0, 2060"):
        # ... [existing code in plot_scenario_1]
        # ... [existing code in plot_scenario_1]

        delta_theta = self.d_theta(self.theta_0, theta_i)
        delta_h = self.d_h(self.h0, hf)

        try:
            i_t_min, i_t_max = map(float, i_t_range.split(','))
            I_t = np.arange(i_t_min, i_t_max, 0.1)
        except Exception as e:
            print("Error in parsing I_t range: ", e)
            return  # Optionally return here to prevent plotting if there's an error        
        t = self.t_for_cum_inf(I_t, delta_theta, delta_h, Ks)
        # write the time to a file
        np.savetxt('time.txt', t, delimiter=',')
        self.new_x = np.arange(round(t[0], 1), round(t[-1], 1), 0.01)

        self.infiltration_rate(self.new_x, t, I_t)
        
        i_t_short = self.short_time_approximation(delta_theta, delta_h, Ks, self.new_x)
        i_t_long = self.long_time_approximation(Ks, self.new_x)

        fig, ax = plt.subplots(figsize=(10, 6))

        # Plot all reference lines for the short time approximation
        
        # Conditionally plot the short time approximation reference lines
        if plot_short_time:
            for ref, params, clr in zip(self.reference_lines, self.ref_line_params, self.ref_line_colors):
                ref_x, ref_y = ref
                ax.plot(ref_x, ref_y, linestyle='-.', alpha=0.6, color=clr)

        # Conditionally plot the infiltration rate reference lines
        if plot_infiltration_rate:
            for ref, params, clr in zip(self.infiltration_rate_ref, self.ref_line_params, self.ref_line_colors):
                ref_x, ref_y = ref
                ax.plot(ref_x, ref_y, alpha=0.6, color=clr, label='hf:{} theta_i:{} Ks:{}'.format(params[0], params[1], params[2]), linewidth=2.5)

        if plot_long_term:
            for ref, params, clr in zip(self.long_time_approximation_ref, self.ref_line_params, self.ref_line_colors):
                ref_x, ref_y = ref
                ax.plot(ref_x, ref_y, linestyle='--', alpha=0.6, color=clr)
    
        ax.plot(self.new_x[1:-5], self.diff[:-5]*100, color='black', label="theta_i=" + str(theta_i) + ", hf=" + str(hf), linewidth=2.5)
        ax.plot(self.new_x, i_t_short, 'black', linestyle='-.', label='Short time approximation')
        ax.plot(self.new_x, i_t_long, 'black', linestyle='--', label='Long time approximation')

        ax.set_ylim(0.1, 140)
        ax.set_xlim(-5, max(self.new_x))
        plt.yscale('log')
        ax.set_xlabel('Time [h]', fontsize=12)
        ax.set_ylabel('infiltration rate [cm/h]', fontsize=12)

                # Parse the x-axis range from the text field
        try:
            x_min, x_max = map(float, x_range.split(','))
            ax.set_xlim(x_min, x_max)
        except Exception as e:
            print("Error in parsing x-axis range: ", e)

        # Move the legend outside of the plot
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

        plt.show()

        print(len(self.short_time_ref))

    def add_new_color(self):
        colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink']
        for color in colors:
            if color not in self.ref_line_colors:
                self.ref_line_colors.append(color)
                return
        print("All colors used up!")

    def on_button_click(self, button):
        
        #add new color for the reference line
        self.add_new_color()
        # Current parameter values
        hf_val = self.hf_slider.value
        theta_i_val = self.theta_i_slider.value
        Ks_val = self.Ks_slider.value

        delta_theta = self.d_theta(self.theta_0, theta_i_val)
        delta_h = self.d_h(self.h0, hf_val)

        try:
            i_t_min, i_t_max = map(float, i_t_range.split(','))
            I_t = np.arange(i_t_min, i_t_max, 0.1)
        except Exception as e:
            print("Error in parsing I_t range: ", e)
            return  # Optionally return here to prevent plotting if there's an error        
        t = self.t_for_cum_inf(I_t, delta_theta, delta_h, Ks_val)
        print(t)
        #self.new_x = np.arange(round(t[0], 1), round(t[-1], 1), 0.01)
        i_t_short = self.short_time_approximation(delta_theta, delta_h, Ks_val, self.new_x)

        # Store the current short time approximation data as a reference line
        self.reference_lines.append((self.new_x, i_t_short))
        self.ref_line_params.append((hf_val, theta_i_val, Ks_val))
        self.short_time_ref.append((self.new_x, i_t_short))
        self.short_time_params.append((hf_val, theta_i_val, Ks_val))

        #append long time approximation
        i_t_long = self.long_time_approximation(Ks_val, self.new_x)
        self.long_time_approximation_ref.append((self.new_x, i_t_long))

        self.diff = self.infiltration_rate(self.new_x, t, I_t)
        self.infiltration_rate_ref.append((self.new_x[1:-5], self.diff[:-5]*100))

        clear_output(wait=True)
        display(self.interactive_plot)
        display(button)

    def create_interactive_plot(self):
        # Create the button
        button = Button(description="Fixate Short time approximation")
        button.on_click(self.on_button_click)

        # Create sliders for the parameters
        self.hf_slider = widgets.FloatSlider(value=-60, min=-200, max=0, step=1, description='hf:')
        self.theta_i_slider = widgets.FloatSlider(value=0.2, min=0, max=1, step=0.01, description='theta_i:')
        self.Ks_slider = widgets.FloatSlider(value=4, min=0, max=10, step=0.1, description='Ks:')

        self.x_range_text = widgets.Text(value="0, 200", description="X Axis Range:")

        self.cum_infiltration_text = widgets.Text(value="4000", description="Cumulative Infiltration:")

         # Adding checkbox widgets
        self.checkbox_short_time = widgets.Checkbox(value=True, description='Short Time Approximation')
        self.checkbox_infiltration_rate = widgets.Checkbox(value=True, description='Infiltration Rate')
        self.checkbox_long_term = widgets.Checkbox(value=False, description='Long Term Approximation')

        self.i_t_range_text = widgets.Text(value="0, 2060", description="I_t Range:")

        # Update interactive plot to include new text field
        self.interactive_plot = interactive(self.plot_scenario_1, 
                                            hf=self.hf_slider, 
                                            theta_i=self.theta_i_slider, 
                                            Ks=self.Ks_slider, 
                                            x_range=self.x_range_text, 
                                            plot_short_time=self.checkbox_short_time, 
                                            plot_infiltration_rate=self.checkbox_infiltration_rate,
                                            plot_long_term=self.checkbox_long_term,
                                            i_t_range=self.i_t_range_text)

        display(self.interactive_plot)
        display(button)



In [25]:
GreenAmptModel(h0=0.40, theta_0=1).create_interactive_plot()

interactive(children=(FloatSlider(value=-60.0, description='hf:', max=0.0, min=-200.0, step=1.0), FloatSlider(…

Button(description='Fixate Short time approximation', style=ButtonStyle())

jeder step in gleicher farbe, anhakeln was geplotted wird, x scale anpassen (bei 0 beginnen) xmax bei user definied, cum inf bis 20000 berechnen dann ist es berechnet

matrix diffusion -> schauen wo script